### 作業目的: 使用N-Gram模型預測文字

本次作業會使用[桃園市官網市政新聞](https://data.gov.tw/dataset/25891)來進行練習

### 載入所需的Libraries

In [175]:
import json
import re
from collections import Counter, namedtuple
from nltk import ngrams

### 載入資料

In [176]:
with open('./WebNews.json', 'r') as f:
    ### <your code> ###
    news_data = json.load(f)

In [177]:
news_data[:2]

[{'file': [],
  'link': [],
  'id': '202006110003',
  'istop': 'N',
  'img': [{'imgname': '0611桃園屏東_200611_0004.jpg',
    'imgcontent': '<br>鄭<br>市<br>長<br>與<br>潘<br>縣<br>長<br>合<br>體<br>推<br>廣<br>防<br>疫<br>互<br>惠<br>旅<br>遊<br>',
    'imgurl': 'http://www.tycg.gov.tw/uploaddowndoc?file=news/202006111817400.jpg&filedisplay=202006111817400.jpg&flag=pic'},
   {'imgname': '0611桃園屏東_200611_0005.jpg',
    'imgcontent': '<br>潘<br>縣<br>長<br>挑<br>選<br>屏<br>東<br>特<br>色<br>伴<br>手<br>禮<br>致<br>贈<br>鄭<br>市<br>長<br>',
    'imgurl': 'http://www.tycg.gov.tw/uploaddowndoc?file=news/202006111817401.jpg&filedisplay=202006111817401.jpg&flag=pic'},
   {'imgname': '0611桃園屏東_200611_0007.jpg',
    'imgcontent': '<br>鄭<br>市<br>長<br>精<br>心<br>準<br>備<br>桃<br>園<br>特<br>色<br>伴<br>手<br>禮<br>送<br>給<br>潘<br>縣<br>長<br>',
    'imgurl': 'http://www.tycg.gov.tw/uploaddowndoc?file=news/202006111817402.jpg&filedisplay=202006111817402.jpg&flag=pic'},
   {'imgname': '0611桃園屏東_200611_0006.jpg',
    'imgcontent': '<br>桃<br>園<br

### 進行資料清洗
觀察上面的資料，資料包含許多其他的資訊，我們需要將真正的新聞內文取出，並且對內文進行文字清洗。
請做以下的文字處理:

1. 取出新聞內文
2. 去除HTML Tags
3. 移除標點符號，只保留英文、數字、中文

In [178]:
#取出新聞內文
corpus_list = [news['detailcontent'] for news in news_data] ### <your code> ###

#去除HTML Tags與標點符號(只保留英文、數字、中文)
corpus_list = [re.sub(r'<[^>]*>|&nbsp|[^(0-9a-zA-Z一-龥)]', '', news) for news in corpus_list]### <your code> ###
len(corpus_list)

1000

### 建立N-Gram模型
N-Gram模型在計算詞機率時為(以Trigram為例)
$$
P(W_i|W_{i-1},W_{i-2}) = \frac{count(W_i,W_{i-1},W_{i-2})}{count(W_{i-1},W_{i-2})}
$$

舉例來說
$$
P(the|this,is) = \frac{count(this\ is\ the)}{count(this\ is)}
$$

In [179]:
def ngram(documents, N=2):
    
    #建立儲存預測字, 所有ngram詞頻字典, 所有字詞(分母)
    ngram_prediction = dict()
    total_grams = list()
    words = list()
    Word = namedtuple('Word', ['word', 'prob']) #使用namedtuple來儲存預測字詞與對應機率
    
    #Word = namedtuple('Word', ['word', 'prob'])
    #x = Word('hello', '0.999')

    for doc in documents:
        # 在每個文章前加上起始(<start>)與結束符號(<end>)
        ### <your code> ###
        split_words = ["<s>"] + list(corpus_list[0]) + ["</s>"]
        
        # 計算分子
        ### <your code> ###
        #ngrams_freq_numerator = ngrams(split_words, N)
        #total_grams += ["".join(nwords) for nwords in ngrams_freq_numerator]        
        [total_grams.append(tuple(split_words[i:i+N])) for i in range(len(split_words)-N+1)]
        
        
        # 計算分母
        ### <your code> ###
        #ngrams_freq_denominator = ngrams(split_words, N-1)
        #words += ["".join(nwords) for nwords in ngrams_freq_denominator]
        [words.append(tuple(split_words[i:i+N-1])) for i in range(len(split_words)-N+2)]

    #計算分子詞頻
    total_word_counter = Counter(total_grams)  ### <your code> ###
    #計算分母詞頻
    word_counter = Counter(words)   ### <your code> ###
    
    #計算所有N-gram預測字詞的機率
    for key in total_word_counter:
        ### <your code> ###
        word = "".join(key[:N-1]) #tuple to str
        if word not in ngram_prediction:
            ngram_prediction.update({word: set()})
        
        next_word_prob = total_word_counter[key] / word_counter[key[:N-1]]
        w = Word(key[-1], f'{next_word_prob}')
        ngram_prediction[word].add(w) #add to set 
        
    return ngram_prediction

### 使用N-Gram模型進行預測
這裡我們使用4 gram模型，也就是輸入三個字之後，第四個字出現的機率，並將輸出依據機率做排序

In [181]:
#建立four_gram模型，並將預測的機率按照大小排列
four_gram_pred = ngram(corpus_list, 4) ### <your code> ###
for word, pred in four_gram_pred.items():
    ### <your code> ###
    sorted_pred = []
    for p in pred:
        sorted_pred.append(p)
    four_gram_pred[word] = sorted_pred

In [182]:
# 執行時因為字典數龐大，顯示會較慢
four_gram_pred

{'<s>迎接': [Word(word='國', prob='1.0')],
 '迎接國': [Word(word='旅', prob='1.0')],
 '接國旅': [Word(word='爆', prob='1.0')],
 '國旅爆': [Word(word='發', prob='1.0')],
 '旅爆發': [Word(word='期', prob='1.0')],
 '爆發期': [Word(word='五', prob='0.5'), Word(word='觀', prob='0.5')],
 '發期五': [Word(word='星', prob='1.0')],
 '期五星': [Word(word='縣', prob='1.0')],
 '五星縣': [Word(word='市', prob='1.0')],
 '星縣市': [Word(word='長', prob='1.0')],
 '縣市長': [Word(word='合', prob='1.0')],
 '市長合': [Word(word='體', prob='1.0')],
 '長合體': [Word(word='推', prob='1.0')],
 '合體推': [Word(word='廣', prob='1.0')],
 '體推廣': [Word(word='桃', prob='1.0')],
 '推廣桃': [Word(word='園', prob='1.0')],
 '廣桃園': [Word(word='屏', prob='1.0')],
 '桃園屏': [Word(word='東', prob='1.0')],
 '園屏東': [Word(word='互', prob='1.0')],
 '屏東互': [Word(word='惠', prob='1.0')],
 '東互惠': [Word(word='旅', prob='1.0')],
 '互惠旅': [Word(word='遊', prob='1.0')],
 '惠旅遊': [Word(word='讓', prob='0.3333333333333333'),
  Word(word='桃', prob='0.3333333333333333'),
  Word(word='並', prob='0.333333333333

In [166]:
#給定字詞，使用ngram預測下一個字的機率(顯示top 10)
text = '鄭文燦'
next_words = ### <your code> ###
for next_word in next_words:
    print('next word: {}, probability: {}'.format(next_word.word, next_word.prob))

SyntaxError: invalid syntax (<ipython-input-166-e1b4ddc12ad3>, line 3)

可自行嘗試使用不同的N搭建不同的N-Gram模型的效果